In [1]:
import altair as alt
import pandas as pd
from IPython.display import Markdown, display, HTML

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/fund_split/"

# SB125 Fund Split Analysis

## Question:
- How did RTAs split SB125 funds between operations and capital?

## Methodology:
- upload all avilable `SB125 fund request template` files to gcs
- examine all files for consistencies:
    - come with cleaning plan for inconsistent examples (files withot capital/operating columns)
- concat all rows across all files


## Notes:
- Some RTPAs did not submit a `SB125 fund request template.xlsx` file, but instead included an equivilent file their allocation package
- There are ~30 files to analyze, most of which need to be uniquely cleaned. All cleaning steps are done in the `fund_split.py` script and saved to GCS

# Read in Melt DF parquet from GCS

In [2]:
all_melt = pd.read_parquet(f"{GCS_PATH}all_fund_requests_melt.parquet")

In [3]:
display(all_melt.shape, all_melt.columns, all_melt.sample(3))

(2296, 8)

Index(['rtpa', 'implementing agenc-y/-ies', 'project', 'fund source',
       'capital/operation fy', 'fund amount', 'project type', 'fiscal year'],
      dtype='object')

,rtpa,implementing agenc-y/-ies,project,fund source,capital/operation fy,fund amount,project type,fiscal year
1507,Merced County Association of Governments (MCAG),"The Bus, TJPA",Microtransit Services Expansion (Vehicles),TIRCP,operating_FY24-25,0,operating,24-25
99,Plumas County Transportation Commission,Plumas County Transportation Commission,Operating Expenses and Free Fares,TIRCP,capital_FY23-24,0,capital,23-24
941,NCTC,Town of Truckee,Keep Truckee Moving: Expansion and Electrifica...,TIRCP,capital_FY26-27,0,capital,26-27


In [4]:
by_source = all_melt.groupby(["fund source"]).agg({"fund amount": "sum", "rtpa": "nunique"}).reset_index()
by_type = all_melt.groupby(["project type"]).agg({"fund amount": "sum",}).reset_index()
by_year = all_melt.groupby(["fiscal year", "project type"]).agg({"fund amount": "sum"}).reset_index()
by_rtpa = all_melt.groupby(["rtpa"]).agg({"fund amount": "sum", "project": "nunique"}).reset_index()


In [19]:
by_type.columns

Index(['project type', 'fund amount'], dtype='object')

# Charts

In [23]:
stack_chart = alt.Chart(by_year).mark_bar(point=True).encode(
    x="fiscal year", 
    y="fund amount", 
    color="project type",
    tooltip=["project type", "fund amount"],
).properties(width=600, height=400)


overall_chart = alt.Chart(by_type).mark_bar(point=True).encode(
    x="project type", 
    y="fund amount", 
    color="project type",
    tooltip=["project type", "fund amount"],
).properties(width=600, height=400)

display(
    Markdown("## Overall split of SB125 allocations funds by capital/operating expenses"),
    overall_chart,
    Markdown("## SB125 allocations funds by capital/operating expenses, by fiscal year"),
    stack_chart,
    #by_year.sort_values(by="project type")
)

## Overall split of SB125 allocations funds by capital/operating expenses

alt.Chart(...)

## SB125 allocations funds by capital/operating expenses, by fiscal year

alt.Chart(...)

# Draft Aggregations

In [12]:
def make_bar(data, x_axis, y_axis):
    chart = (
        alt.Chart(data)
        .mark_bar()
        .encode(
            x=x_axis,
            y=y_axis,
            #color= color_val,
        )
        .properties(width="container", height=400)
    )
    text = chart.mark_text(align="left", baseline="middle", dx=3).encode(
            text="fund amount"
    )
    return chart + text

In [13]:
display(by_type)
make_bar(by_type, y_axis="project type", x_axis="fund amount")

,project type,fund amount
0,capital,1855348721
1,operating,625654847


alt.LayerChart(...)

In [14]:
display(by_year)
make_bar(
    by_year,
    y_axis="fiscal year",
    x_axis="fund amount",
)

,fiscal year,project type,fund amount
0,23-24,capital,945242943
1,23-24,operating,230807266
2,24-25,capital,812480147
3,24-25,operating,169494114
4,25-26,capital,49092309
5,25-26,operating,96177774
6,26-27,capital,48533322
7,26-27,operating,129175693


alt.LayerChart(...)

In [15]:
alt.Chart(by_year).mark_line(point=True).encode(
    x="fiscal year", y="fund amount", color="project type"
).properties(width=600, height=400)

alt.Chart(...)

In [16]:
make_bar(by_source, y_axis="fund source", x_axis="fund amount")

alt.LayerChart(...)

In [17]:
make_bar(by_rtpa, "rtpa", "fund amount")

alt.LayerChart(...)